In [2]:
%load_ext sagemaker_studio_analytics_extension.magics
%sm_analytics emr connect --verify-certificate False --cluster-id j-2B2F2J67GX6CK --auth-type None

Successfully read emr cluster(j-2B2F2J67GX6CK) details
Initiating EMR connection..
Starting Spark application


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.
{"namespace": "sagemaker-analytics", "cluster_id": "j-2B2F2J67GX6CK", "error_message": null, "success": true, "service": "emr", "operation": "connect"}


In [3]:
import pyspark
from functools import reduce
from operator import add
from pytz import timezone
from pyspark import  StorageLevel, SparkContext, SparkConf
import pyspark.sql.types  as st
import pyspark.sql.functions  as func
from pyspark.sql.functions import *
from pyspark.sql.window import Window as W
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from datetime import datetime, timedelta
from pyspark.sql.types import *
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T
from datetime import datetime, timedelta


conf = SparkConf()
spark_cfgs = {
    "spark.app.name": "MMIX - Actemra ETL",
    "spark.hadoop.hive.exec.dynamic.partition.mode": "nonstrict",
    "spark.spark.sql.legacy.timeParserPolicy": "LEGACY",
    "spark.sql.parquet.compression.codec": "SNAPPY",
    "spark.sql.sources.partitionOverwriteMode": "dynamic",
}

for k, v in spark_cfgs.items():
    conf.set(k, v)
spark = (SparkSession.builder.enableHiveSupport().config(conf=conf).getOrCreate()
)
spark

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
table_paths = [
    ('mars_actemra_net_sales', 's3://cmg-oasis-prod-sandbox-measurement-analyst/mars_actemra_outputs/mars_actemra_net_sales/'),
    ('mars_actemra_gne_sales_staging', 's3://cmg-oasis-prod-sandbox-measurement-analyst/mars_actemra_outputs/mars_actemra_gne_sales_staging/'),
    ('mars_sales_product_actemra_staging', 's3://cmg-oasis-prod-sandbox-measurement-analyst/mars_actemra_outputs/mars_sales_product_actemra_staging/'),
    ('mars_actemra_tam_staging', 's3://cmg-oasis-prod-sandbox-measurement-analyst/mars_actemra_outputs/mars_actemra_tam_staging/'),
    ('mars_xo_actemra_staging', 's3://cmg-oasis-prod-sandbox-measurement-analyst/mars_monthly_outputs/mars_xo_actemra_staging/')
]


for table_name, input_path in table_paths:
    spark.sql(f"""
    CREATE TABLE IF NOT EXISTS sandbox_measurement_analyst.{table_name}
    USING PARQUET
    LOCATION '{input_path}'
    """)
    print(f"Table {table_name} created")

print("All tables complete")


In [4]:
df_prepivot = spark.sql("""
    with combined_data as (
        select distinct
            product_brand_name,
            year_month,
            zip as zip_code,
            audience,
            channel,
            reach,
            engage,
            cost
        from
            sandbox_measurement_analyst.mars_xo_actemra_staging
        union all
        select distinct
            product_brand_name, 
            year_month,
            zip as zip_code,
            audience,
            channel,
            reach,
            null as engage,
            cost
        from
            sandbox_measurement_analyst.mars_actemra_tam_staging
    )

    select * from combined_data
""")
df_prepivot.createOrReplaceTempView("combined")

# df_pre_pivot.write.mode("overwrite").parquet(
#     f"s3://cmg-oasis-prod-sandbox-measurement-analyst/mars_monthly_data/prepivot/{today}/"
# )

# spark.sql(f"""
#     CREATE TABLE IF NOT EXISTS sandbox_measurement_analyst.mars_pre_pivot_monthly
#     USING PARQUET
#     LOCATION 's3://cmg-oasis-prod-sandbox-measurement-analyst/mars_monthly_data/prepivot/{today}/'
# """)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
today = datetime.today().strftime('%Y%m%d')
df_pivoted = (
    spark.sql(
"""
    SELECT
        product_brand_name,
        zip_code,
        year_month,
        channel || '_' || audience AS channel_audience,
        reach,
        engage,
        cost
    FROM combined
"""
    )
    .groupBy(
        [
            "product_brand_name",
            "zip_code",
            "year_month"
        ]
    )
    .pivot("channel_audience")
    .sum("reach", "engage", "cost")
)

df_pivoted = df_pivoted.select(
    *[
        F.col(col).alias(col.replace("_sum(", "_").replace(")", ""))
        for col in df_pivoted.columns
    ]
)
df_pivoted.registerTempTable('post_pivot')

# df_pivoted.write.mode("overwrite").parquet(
#     f"s3://cmg-oasis-prod-sandbox-measurement-analyst/mars_actemra_outputs/postpivot/{today}/"
# )

# spark.sql(f"""
#     CREATE TABLE IF NOT EXISTS sandbox_measurement_analyst.mars_postpivot_monthly_actemra
#     USING PARQUET
#     LOCATION 's3://cmg-oasis-prod-sandbox-measurement-analyst/mars_actemra_outputs/postpivot/{today}/'
# """)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[]

In [ ]:
df_final = spark.sql(
"""
    with cte as (
        select distinct
            coalesce(
                pivoted.product_brand_name,
                sales.product_brand_name,
                hcp_count.product_brand_name, 
                net.product_brand_name
            ) as product_brand_name,
            coalesce(pivoted.zip_code, sales.zip, hcp_count.zip_code) as zip_code,
            coalesce(pivoted.year_month, sales.year_month, net.year_month) as date_year_month,
              alert_hcp_reach , 
              alert_hcp_engage , 
              alert_hcp_cost , 
              display_dtc_reach , 
              display_dtc_engage , 
              display_dtc_cost , 
              display_hcp_reach , 
              display_hcp_engage , 
              display_hcp_cost , 
              enewsletter_hcp_reach , 
              enewsletter_hcp_engage , 
              enewsletter_hcp_cost , 
              house_email_hcp_reach , 
              house_email_hcp_engage , 
              house_email_hcp_cost , 
              paid_search_dtc_reach , 
              paid_search_dtc_engage , 
              paid_search_dtc_cost , 
              paid_search_hcp_reach , 
              paid_search_hcp_engage , 
              paid_search_hcp_cost , 
              tam_hd_ce_reach , 
              tam_hd_ce_engage , 
              tam_hd_ce_cost , 
              third_party_email_hcp_reach , 
              third_party_email_hcp_engage , 
              third_party_email_hcp_cost ,
            sales.sales_qty as gne_867_sales_qty_eqv,
            net.net_sales, 
            hcp_count.hcp_count
        from post_pivot as pivoted
        full outer join
            sandbox_measurement_analyst.mars_actemra_gne_sales_staging as sales
            on
                pivoted.product_brand_name = sales.product_brand_name
                and pivoted.year_month = sales.year_month
                and pivoted.zip_code = sales.zip
        full outer join
            sandbox_measurement_analyst.mars_hcp_count_staging as hcp_count
            on
                pivoted.product_brand_name = hcp_count.product_brand_name
                and pivoted.zip_code = hcp_count.zip_code
        full outer join 
            sandbox_measurement_analyst.mars_actemra_net_sales as net 
            on pivoted.product_brand_name = net.product_brand_name
            and pivoted.year_month = net.year_month 

    )
    select *
    from cte 
    where product_brand_name = 'ACTEMRA'
    and date_year_month between '202201' and '202312'
""")

In [ ]:
output_path = f's3://cmg-oasis-prod-sandbox-measurement-analyst/mars_actemra_outputs/mars_actemra_combined_pivoted/'
today = datetime.today().strftime('%Y%m%d')
table_name = 'mmix_combined_pivoted_actemra'

df_final.write.mode("overwrite").parquet(
    f"{output_path}{today}/"
)
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS sandbox_measurement_analyst.{table_name}
    USING PARQUET
    LOCATION '{output_path}{today}/'
""")
